# 0. Install Dependencies

In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2
!pip install pygame
!pip install requests

# 1. Test Random Environment with OpenAI Gym

In [1]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import pygame
import os
import string
import requests
import json

C:\Users\rodam\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\rodam\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\rodam\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


pygame 2.0.1 (SDL 2.0.14, Python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
window_width, window_height = 1000, 500

In [3]:
class RealEstateEnv(Env):
    def __init__(self, houses):
        # Actions we can take, lower price, same price, increase price
        self.action_space = Discrete(3)
        # House prices array
        self.observation_space = Box(low=np.array([100000]), high=np.array([1100000]))
        # Set state
        self.state = -1000
        # Set finding length
        self.finding_length = 180
        
        self.pos = random.randint(0, len(houses)-1)
        self.house = houses[self.pos]
        self.num_properties = 5
        
        self.best_house = self.house
        self.best_state = self.state
    
    def init_render(self):
        pygame.init()
        self.window = pygame.display.set_mode((window_width, window_height))
        pygame.display.set_caption("Real Estate Env")
        self.clock = pygame.time.Clock()
    
    
    def text(self, text):
        font_color=(0,150,250)
        font_obj=pygame.font.Font(None, 25)
        # Render the objects
        text_obj = []
        for t in text:
            text_obj.append(font_obj.render(t, True, font_color))
        return text_obj
        
    def step(self, action, customer, houses):
        
        
        # Apply action
        if action == 0 and self.pos != 0:
            self.pos -= 1
        elif action == 2 and self.pos != len(houses)-1:
            self.pos += 1
        
        self.house = houses[self.pos]
        
        weights = [1/16, 3/16, 3/16, 1/2, 1/16]
        
        bedroom_dif = (self.house.bedrooms / customer.num_bedrooms)*weights[0]
        square_footage_dif1 = (self.house.lot_sqft / customer.lot_sqft)*weights[1]
        square_footage_dif2 = (self.house.house_sqft / customer.house_sqft)*weights[2]
        price_dif = (self.house.price / customer.price)*weights[3]
        bathroom_dif = (self.house.bathrooms / customer.num_bathrooms)*weights[4]
        
        state2 = bedroom_dif + square_footage_dif1 + price_dif + square_footage_dif2 + bathroom_dif
        
        # Reduce house finding length by 1 second
        self.finding_length -= 1
        
        # Calculate reward
        if abs(1 - self.state) > abs(1 - state2): 
            self.state = state2
            reward = 1
            if abs(1 - self.best_state) > abs(1 - self.state):
                self.best_state = self.state
                self.best_house = houses[self.pos]
        else:
            reward = -1
        
        # Check if finding time is over
        if self.finding_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply price fluctuations
        # self.state += random.randint(-10000,25000)
        # Set placeholder for info
        info = {}
        
        self.state = round(self.state, 3)
        self.best_state = round(self.best_state, 3)
        
        # Return step information
        return self.state, reward, done, info

    def render(self, text_obj):
        # Implement viz
        self.window.fill((0,0,0))
        
        # score
        for i in range(5):
            self.window.blit(text_obj[i],(i*160, 0))
        
        # other properties
        count = 5
        for i in range(self.num_properties):
            for j in range(3):
                self.window.blit(text_obj[count], (200*i, j*50+50))
                count += 1
        
        #left person
        #head
        pygame.draw.circle(self.window, (66, 84, 81), (int(150), int(320)), 40)
        #eyes
        pygame.draw.circle(self.window, (0, 200, 200), (int(130), int(310)), 8)
        pygame.draw.circle(self.window, (0, 200, 200), (int(170), int(310)), 8)
        #body
        pygame.draw.rect(self.window, (45,45,45), [120, 360, 60, 140])
        pygame.draw.line(self.window, (55, 55, 55), [0, 30],[window_width,30], 3)
        pygame.draw.line(self.window, (55, 55, 55), [0, 180],[window_width,180], 3)
        
        pygame.display.update()
        
    
    def reset(self):
        # Reset base score
        self.state = -1000
        # Reset finding time
        self.finding_length = 180
        self.pos = random.randint(0, len(houses)-1)
        self.house = houses[self.pos]
        return self.state

In [4]:
class Customer():
    def __init__(self, num, price):
        self.price = price
        self.lot_sqft = random.randint(500, 2000)
        self.house_sqft = int(self.lot_sqft * random.uniform(0.25, 9))
        self.num_bedrooms = random.randint(1, 4)
        self.num_bathrooms = random.randint(5, 20)
        self.id = num
        self.color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)]
    
    def render(self, env):
        #right person
        #head
        pygame.draw.circle(env.window, (self.color[0],self.color[1],self.color[2]), (window_width-int(150), int(320)), 40)
        #eyes
        pygame.draw.circle(env.window, (0, 20, 20), (window_width-int(130), int(310)), 8)
        pygame.draw.circle(env.window, (0, 20, 20), (window_width-int(170), int(310)), 8)
        #mouth
        pygame.draw.line(env.window, (255, 205, 205), [window_width-int(135), int(338)],[window_width-int(165),int(338)], 6)
        #body
        pygame.draw.rect(env.window, (self.color[0],self.color[1],self.color[2]), [window_width-180, 360, 60, 140])
        pygame.display.update()
    
    @staticmethod
    def load_customers(x):
        arr = []
        
        for i in range(x):
            arr.append(Customer(i + 1))
        
        return arr

In [5]:
class Houses():
    def __init__(self, properties):
        self.lot_sqft = int(properties[0])
        self.bedrooms = int(properties[1])
        self.price = int(properties[2])
        self.bathrooms = int(properties[3])
        self.house_sqft = int(properties[4])

In [6]:
# api data
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

access_key = input("Server Token: ")
data_api = requests.get("https://api.bridgedataoutput.com/api/v2/OData/test/Property?access_token=" + access_key)
lot_sizes_sqft = []
house_sqft = []
bedrooms_total = []
list_prices = []
bathrooms = []

for i in range(10):
    total_houses = data_api.json()['value']

    for item in total_houses:
        t1 = item['LotSizeSquareFeet']
        lot_sizes_sqft.append(t1)
        t2 = item['BedroomsTotal']
        bedrooms_total.append(t2)
        t3 = item['ListPrice']
        list_prices.append(t3)
        t4 = item['LivingArea']
        house_sqft.append(t4)
        t5 = item['BathroomsTotalInteger']
        bathrooms.append(t5)
    data_api = requests.get(data_api.json()['@odata.nextLink'])

houses = []
num_properties = 5
for i in range(len(lot_sizes_sqft)):
    temp2 = []
    temp2.append(lot_sizes_sqft[i])
    temp2.append(bedrooms_total[i])
    temp2.append(list_prices[i])
    temp2.append(bathrooms[i])
    temp2.append(house_sqft[i])
    house = Houses(temp2)
    houses.append(house)

houses = sorted(houses, key=lambda x: x.price)

num = 1
for house in houses:
    print("House " + str(num))
    print("Price: {} Lot Square Footage: {} Bedrooms: {} Bathrooms: {} House Square Footage: {}".format(house.price, house.lot_sqft, house.bedrooms, house.bathrooms, house.house_sqft))
    num += 1
    print()

Server Token: b9fb97d623d73441e25ee82bfcb8c4fa
House 1
Price: 45625 Lot Square Footage: 521 Bedrooms: 2 Bathrooms: 16 House Square Footage: 2080

House 2
Price: 49499 Lot Square Footage: 841 Bedrooms: 1 Bathrooms: 16 House Square Footage: 4284

House 3
Price: 56610 Lot Square Footage: 1936 Bedrooms: 1 Bathrooms: 18 House Square Footage: 5182

House 4
Price: 97033 Lot Square Footage: 1293 Bedrooms: 3 Bathrooms: 10 House Square Footage: 2660

House 5
Price: 99365 Lot Square Footage: 1390 Bedrooms: 4 Bathrooms: 17 House Square Footage: 8247

House 6
Price: 105162 Lot Square Footage: 1365 Bedrooms: 3 Bathrooms: 10 House Square Footage: 3341

House 7
Price: 120978 Lot Square Footage: 1665 Bedrooms: 4 Bathrooms: 9 House Square Footage: 1175

House 8
Price: 191107 Lot Square Footage: 991 Bedrooms: 1 Bathrooms: 17 House Square Footage: 553

House 9
Price: 194184 Lot Square Footage: 1407 Bedrooms: 4 Bathrooms: 12 House Square Footage: 8131

House 10
Price: 198400 Lot Square Footage: 1603 Bedroo

In [7]:
env = RealEstateEnv(houses)
env.init_render()

C:\Users\rodam\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [8]:
env.observation_space.sample()

array([895833.6], dtype=float32)

In [ ]:
episodes = 10
num_customers = 10
customers = []
customers.append(Customer(1, 500000))
pygame.init()
commission = 0
goal = 100000
num_customer = 1
while commission < goal:
    env.best_state = env.state
    env.best_house = env.house
    for episode in range(1, episodes+1):
        state = env.reset()
        done = False
        score = 0

        while not done:
            textArray = ["Fit: " + str(env.state),
                         "Best Fit: " + str(env.best_state),
                         "Customer ID: " + str(customers[num_customer-1].id),
                         "Episode: " + str(episode),
                         "Commissions: " + str(commission),
                         "Curr Price: " + str(env.house.price), 
                         "Goal Price: " + str(customers[num_customer-1].price), 
                         "Best Price: " + str(env.best_house.price), 
                         "Curr HFootage: " + str(env.house.house_sqft), 
                         "Goal HFootage: " + str(customers[num_customer-1].house_sqft), 
                         "Best HFootage: " + str(env.best_house.house_sqft), 
                         "Curr Bedrooms: " + str(env.house.bedrooms), 
                         "Goal Bedrooms: " + str(customers[num_customer-1].num_bedrooms),
                         "Best Bedrooms: " + str(env.best_house.bedrooms), 
                         "Curr Bathrooms: " + str(env.house.bathrooms),
                         "Goal Bathrooms: " + str(customers[num_customer-1].num_bathrooms),
                         "Best Bathrooms: " + str(env.best_house.bathrooms), 
                         "Curr LFootage: " + str(env.house.lot_sqft),
                         "Goal LFootage: " + str(customers[num_customer-1].lot_sqft),
                         "Best LFootage: " + str(env.best_house.lot_sqft), 
                        ]
            text_obj = env.text(textArray)
            env.clock.tick(30)
            action = env.action_space.sample()
            n_state, reward, done, info = env.step(action, customers[num_customer-1], houses)
            score+=reward
            env.render(text_obj)
            customers[num_customer-1].render(env)
            for eve in pygame.event.get():
                if eve.type==pygame.QUIT:
                    pygame.quit()
                    #sys.exit()
        print('Customer ID: {} Episode:{} Score:{}'.format(customers[num_customer-1].id, episode, score))
        
    num_customer += 1

    # commission code
    commission += env.best_house.price * 0.03
    difference = abs(1 - env.best_state)
    new_price = 0
    if difference < 0.01:
        new_price = env.best_house.price * (1.1 - difference)
    else:
        new_price = env.best_house.price * (1 - difference)
    customers.append(Customer(num_customer, new_price))
        

pygame.quit()

Customer ID: 1 Episode:1 Score:-158
Customer ID: 1 Episode:2 Score:-148
Customer ID: 1 Episode:3 Score:-160
Customer ID: 1 Episode:4 Score:-154
Customer ID: 1 Episode:5 Score:-174
Customer ID: 1 Episode:6 Score:-158
Customer ID: 1 Episode:7 Score:-174
Customer ID: 1 Episode:8 Score:-144
Customer ID: 1 Episode:9 Score:-162


# 2. Create a Deep Learning Model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
actions

In [ ]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
del model 

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

# 3. Build Agent with Keras-RL

In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)